In [5]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
# unimodal network of who-knows-who
## diary= DataFrame -- from hand typed csv
## path= the storage path
## export a csv of nodes, a csv of adjacency matrix, and a csv of edges
## return
def ppl(diary):
    ## clean the data
    diary.dropna(axis=[0,1],how='all',inplace=True)
    diary['Participants']=StdNm(diary['Participants'])
    
    ## make people-event DataFrame
    ppl_event=diary[['Type','Participants']].dropna(subset=['Participants'])
    
    #### make event index
    event_iter=0
    event_idx=[]
    for boo in ppl_event['Type'].notnull():
        if boo:
            event_iter+=1
        event_idx.append(event_iter)
    
    ppl_event["Event"]=event_idx
    
    ## make a series of unique names as the index and columns of the adjacency matrix
    ppl_uniq=pd.Series(ppl_event['Participants'].value_counts().index).append(pd.Series("謝蘭生"),ignore_index=True)
    ppl_df=pd.DataFrame(ppl_uniq).set_index(0)
    
    ## initialize the adjacency matrix
    AdjMa=ppl_df.join(ppl_df.T,how='outer').fillna(0)
    
    ## Count frequency from event
    event=ppl_event.set_index('Event').drop('Type',axis=1)
    for i in np.arange(1,event_iter):
        if len(event.ix[i])>1:
            idx=event.ix[i]['Participants']
            AdjMa.ix[idx,idx]+=1
        i+=1
    #### diagonal -- 0
    np.fill_diagonal(AdjMa.values,0)
    #### for 謝蘭生
    freq=np.append(ppl_event['Participants'].value_counts().values,0)
    AdjMa["謝蘭生"]=freq
    AdjMa.ix["謝蘭生"]=freq
    ## export a csv of Adjacency Matrix
    AdjMa.to_csv('pplAdj.csv')
    
    ## export a csv of people as nodes
    ppl_df=ppl_df.reset_index().reset_index().rename(columns={0:"Label","index":"Id"})
    ppl_df.to_csv('pplNodes.csv',index=False)
    
    ## make edgelist
    AdjMa=AdjMa.replace(0,np.nan)
    
    i=0
    Source=[]
    Target=[]
    Weight=[]
    for colName,ser in AdjMa.iteritems():
        # symmetric matrix --> triangle
        prep=ser[:i][pd.notnull(ser)]
        for idx,value in prep.iteritems():
            #print colName,idx,value
            Source.append(colName)
            Target.append(idx)
            Weight.append(value)
        i+=1
    
    #source=pd.merge(pd.DataFrame(Source),ppl_df,how='left',left_on=0,right_on="Label")["Id"]
    #target=pd.merge(pd.DataFrame(Target),ppl_df,how='left',left_on=0,right_on="Label")["Id"]
    edge=pd.DataFrame({"Id":np.arange(len(Weight)),"Source":source,"Target":target,\
                       "Type":len(Weight)*["Undirected"],"Weight":Weight})
    edge.to_csv('pplEdges.csv',index=False)
    
    return

In [3]:
# people-place bimodal network
## diary = DataFrame -- from hand typed csv
## path = the storage path
## export a csv of people-place nodes and a csv of people-place edges
## return
def ppl_plc(diary):
    
    ## clean the data
    diary.dropna(axis=[0,1],how='all',inplace=True)
    diary['Participants']=StdNm(diary['Participants'])
    
    #### make nodes
    ppl_plc=diary[["Place","Participants"]].dropna()
    ppl=pd.DataFrame({"Label":ppl_plc["Participants"].unique(),"Modularity Class":"People"})
    plc=pd.DataFrame({"Label":ppl_plc["Place"].unique(),"Modularity Class":"Place"})
    binodes=pd.DataFrame(ppl.append(plc,ignore_index=True))
    binodes=binodes.reset_index().rename(columns={"index":"Id"})
    binodes.to_csv("biNodes.csv",index=False)
    
    #### make edges
    target=pd.merge(ppl_plc,binodes,how='left',left_on="Place",right_on="Label")["Id"]
    source=pd.merge(ppl_plc,binodes,how='left',left_on="Participants",right_on="Label")["Id"]
    edges=pd.DataFrame({"Id":np.arange(len(source)),"Source":source,"Target":target,"Weight":1})
    edges=edges.groupby(["Source","Target"]).sum().reset_index()
    edges.to_csv("ppl_plc_Edges.csv",index=False)
    
    return

In [6]:
# Standardize People's Names -- One to Many Transformation
## arg: nonstd = Series of non-standard names
## return standardized names in a Series
def StdNm (nonstd):
    
    ## make a DataFrame indexed by standardized name and contains columns "FirstName", "OtherNames", "OtherNames1"
    std=pd.read_csv("StandardNames.csv")
    std["FullName"]=std["LastName"]+std["FirstName"]
    std["FullName"].fillna(method="ffill",inplace=True)
    std.set_index("FullName",inplace=True)
    std["OtherNames1"]=std["LastName"].fillna(method="ffill")+std['OtherNames']
    std.drop(["Details","Studio","LastName"],axis=1,inplace=True)
    
    ## make a dictionary of {key=standardized name: value=np.array of alternative names}
    dNames={}
    for stdNm in std.index.unique():
        if np.ndim(std.ix[stdNm])==1:
            arr=std.ix[stdNm].values
            dNames[stdNm]=arr[pd.notnull(arr)]
        else:
            arr=std.ix[stdNm].stack().values
            dNames[stdNm]=arr[pd.notnull(arr)]
    
    ## Standardize names in the given Series
    for idx, nonstdNm in nonstd.dropna().iteritems():
        for stdNm in dNames.keys():
            for alt in dNames[stdNm]:
                if (alt in nonstdNm) & (nonstdNm not in dNames.keys()):
                    nonstd[idx]=nonstdNm.replace(alt,stdNm)
    
    return nonstd